Credits

https://github.com/yunjey/pytorch-tutorial

In [1]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn

Basic autograd example

In [7]:
# only floating type of tensors can require gradients
# Create tensors.
x = torch.tensor(2., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * pow(x,2) + b*20    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients, x.grad means derivate of y wrt to x
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(8.)
tensor(4.)
tensor(20.)


Autograd example 2

In [16]:
x=torch.randn(10,3)
y=torch.randn(10,2)

linear=nn.Linear(3,2)
print('w:',linear.weight)
print('b:',linear.bias)

criterion=nn.MSELoss()
optimizer=torch.optim.SGD(linear.parameters(),lr=0.01)

pred=linear(x)

loss=criterion(pred,y)
print('loss:',loss.item())

loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

print('weights after 1 step optimization: ',linear.weight)
print('bias after 1 step optimization: ',linear.bias)

w: Parameter containing:
tensor([[ 0.0845, -0.1800, -0.0152],
        [-0.4147, -0.1513, -0.1084]], requires_grad=True)
b: Parameter containing:
tensor([0.1355, 0.0074], requires_grad=True)
loss: 1.7408394813537598
dL/dw:  tensor([[ 0.5634, -0.1608,  0.3922],
        [-0.8100, -0.0962,  0.5481]])
dL/db:  tensor([0.3663, 0.0902])
loss after 1 step optimization:  1.7248921394348145
weights after 1 step optimization:  Parameter containing:
tensor([[ 0.0789, -0.1784, -0.0191],
        [-0.4066, -0.1504, -0.1139]], requires_grad=True)
bias after 1 step optimization:  Parameter containing:
tensor([0.1318, 0.0065], requires_grad=True)


A proof that how this works

In [20]:
a=[[ 0.0845, -0.1800, -0.0152],
        [-0.4147, -0.1513, -0.1084]]

b=[[ 0.5634, -0.1608,  0.3922],
        [-0.8100, -0.0962,  0.5481]]

a=np.array(a)
b=np.array(b)
c=a - 0.01*b

c

array([[ 0.078866, -0.178392, -0.019122],
       [-0.4066  , -0.150338, -0.113881]])

Loading data from numpy

In [21]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

Input pipeline

In [ ]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass


In [ ]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


In [ ]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)


Saving model and extension is ckpt

In [ ]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))